## Custom Agent using langchain creation
## Using OpenAI to make parallel Function calling

In this notebook teach you how to create custom agent with different tools


# Custom Tools creation using Langchain

In [1]:
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o",temperature=0.9,max_tokens=500
)

@tool
def SqlQueryWritter(user_input : str)->str:
    """
    A tool that generates SQL queries based on user input.
    """
   
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system","you are an helpfull assistent to write the SQL Queries"),
        
            ("user","{query}")
        ]
    )

    chain = prompt | llm 

    response = chain.invoke({"query":user_input})

    return response.content



@tool
def SqlQuerySummarizer(user_input : str)->str:
    """
    A tool that generates SQL queries based on user input.
    """
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system","you are an helpfull assistent , Summarize the SQL Queries"),
        
            ("user","{query}")
        ]
    )

    chain = prompt | llm 

    response = chain.invoke({"query":user_input})

    return response.content


In [2]:
tools=[SqlQueryWritter,SqlQuerySummarizer]


In [3]:
# Define the tools
#tools = [SqlQueryWritter]
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
# Create the prompt for the agent
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful assistant to write queries or summarize queries"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)


In [8]:
# Optional 

from langchain.agents import load_tools                                         
from langchain.agents import initialize_agent

chat_history = []

agent = initialize_agent(
    tools=tools,
    llm=llm,
    prompt=prompt 
)


In [4]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
llm_withtools=llm.bind_tools(tools)
agent=(
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_withtools
    | OpenAIToolsAgentOutputParser()
)

In [5]:
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferWindowMemory
#memory = ConversationBufferWindowMemory(memory_key="chat_history",input_key="input",output_key='output', return_messages=True, k=4)


agent_executor = AgentExecutor(agent=agent, tools=tools)
                               

In [6]:
res=agent_executor.invoke({"input":"top 20 records sql sql query"})

In [7]:
print(res["output"])

To retrieve the top 20 records from a table in SQL, you can use the `LIMIT` clause in SQL. However, the syntax might vary slightly depending on the database system you are using. Here are examples for a few common SQL databases:

### MySQL, PostgreSQL, SQLite:
```sql
SELECT *
FROM your_table_name
ORDER BY some_column
LIMIT 20;
```

### SQL Server:
```sql
SELECT TOP 20 *
FROM your_table_name
ORDER BY some_column;
```

### Oracle:
In Oracle, you can use the `FETCH FIRST` clause (available in Oracle 12c and later):

```sql
SELECT *
FROM your_table_name
ORDER BY some_column
FETCH FIRST 20 ROWS ONLY;
```

Or use a subquery with `ROWNUM`:

```sql
SELECT *
FROM (
    SELECT *
    FROM your_table_name
    ORDER BY some_column
)
WHERE ROWNUM <= 20;
```

### Notes:
- Replace `your_table_name` with the name of your table.
- Replace `some_column` with the column you want to sort by if ordering is necessary; otherwise, you can leave out the `ORDER BY` clause if the order of records doesn't matter t

## Vector DB


In [8]:

from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# Load url
loaders=UnstructuredURLLoader(urls=[
    "https://www.icicibank.com/personal-banking/cards/credit-card",
    "https://www.hdfcbank.com/personal/pay/cards/credit-cards",
])

data=loaders.load()

# splitting chunkingour data
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)
docs=text_splitter.split_documents(data)

# OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Vector index creation with FAISS
vector_index=FAISS.from_documents(docs,embeddings)

# Save vector index locally
vector_index.save_local("credicard")




## Load Vector DB

In [9]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import OpenAI

# Load Vector
vectordb=FAISS.load_local("credicard",OpenAIEmbeddings(),allow_dangerous_deserialization=True)
# retriever
retrieverdb=vectordb.as_retriever(search_kwargs={'k': 2})

llm = OpenAI(temperature=0.9,max_tokens=400)

# Create chain
chain=RetrievalQAWithSourcesChain.from_chain_type(OpenAI(temperature=0), chain_type="stuff", retriever=retrieverdb)



In [36]:
response=chain.invoke({
    "question": "Pixel Play Credit Card The Born Digital Range of Credit Cards? provide benifits?"
})

In [37]:
response

{'question': 'Pixel Play Credit Card The Born Digital Range of Credit Cards? provide benifits?',
 'answer': " The Pixel Play Credit Card from HDFC Bank's Born Digital Range of Credit Cards provides benefits such as 5% cashback on select categories, 3% cashback on a choice of one e-commerce merchant, and 1% unlimited cashback on all spends. It also allows for customization of the card design and billing cycle. \n",
 'sources': 'https://www.hdfcbank.com/personal/pay/cards/credit-cards'}

In [7]:
response['answer']

' The benefits of the Freedom Credit Card include complimentary Times Prime Annual Membership, up to 20% discount offers on travel and shopping, 25% discount on movies, and reward points for every Rs.150 spent. Other popular HDFC Bank Credit Cards include the All Miles Credit Card, Bharat Credit Card, and Diners Club Premium Credit Card. \n'

## Normal Method Function call using OpenAI

In [12]:
import json
from datetime import datetime,timedelta
from openai import OpenAI

openai=OpenAI()


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_fancy_names",
            "description": "This question will ask user to answered to llm.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "This question will ask user to answered to llm .",
                    },
                },
                "required": ["question"],
                
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_creditcard_info",
            "description": "This funtion will return the credit card details  informations",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "question": "This question will ask user to answered to llm .",
                    },
                },
                "required": ["question"],
            },
        }
    },


]


def get_fancy_names(question :str):

    """Get flight information between two locations."""
    response=openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": question},
        ],
        max_tokens=100,
        temperature=0.7,
        n=1,
    )
    return json.dumps({"company_names":response.choices[0].message.content})

    #return json.dumps({"name":"karuppu"})

def get_creditcard_info(question :str):
    # # Load Vector
    # vectordb=FAISS.load_local("credicard",OpenAIEmbeddings(),allow_dangerous_deserialization=True)
    # # retriever
    # retrieverdb=vectordb.as_retriever(search_kwargs={'k': 2})

    # #llm = OpenAI(temperature=0.9,max_tokens=400)

    # # Create chain
    # chain=RetrievalQAWithSourcesChain.from_chain_type(OpenAI(), chain_type="stuff", retriever=retrieverdb)

    response=chain.invoke({
    "question": question
    })

    response =json.dumps(response['answer'])

    return response

#user_prompt = "Give me the 5 fancy names of the pencil company?"
user_prompt="Pixel Play Credit Card The Born Digital Range of Credit Cards? provide benifits offers?"
#user_prompt = "give me the 5 names of company which makes products of pencil? "



# messages.append({"role": "user", "content": user_prompt})
output = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role" : "system",
            "content" : " You are the helpfull assistence to give me the better results"
        },
        {
            "role" : "user",
            "content" : " user will ask questions which might be present in the llm or vector database"
        },
        {
            "role" : "assistant",
            "content" : "Hi there! I can help with that. Can you please provide more info?"
        },
        {
            "role" : "user",
            "content" : user_prompt
        }
    ],
    tools=tools
)



In [95]:
## Learniing samples

# json.loads(output.choices[0].message.tool_calls[0].function.arguments)
# output.choices[0].message.tool_calls[0].function.name

# import json
# creadit_card_info=json.loads(output.choices[0].message.tool_calls[0].function.arguments)
# #kpparent_info

# card_function=eval(output.choices[0].message.tool_calls[0].function.name)
# pixel_play_card = card_function(**creadit_card_info)

# pixel_play_card #we will get parent details with helo of llm

In [13]:
llm_response=output.choices[0].message.tool_calls
# Making which function will call
if llm_response:
    function_name=output.choices[0].message.tool_calls[0].function.name
    if function_name == "get_creditcard_info":
        creadit_card_info=json.loads(output.choices[0].message.tool_calls[0].function.arguments)
        card_function=eval(output.choices[0].message.tool_calls[0].function.name)
        pixel_play_card = card_function(**creadit_card_info)
        print(pixel_play_card)

    elif function_name == "get_fancy_names":
        fancy_info=json.loads(output.choices[0].message.tool_calls[0].function.arguments)
        get_fancy_function=eval(output.choices[0].message.tool_calls[0].function.name)
        fancy_names = get_fancy_function(**fancy_info)
        print(fancy_names)

#if name == "get_creditcard_info" :
else:
    print("Error while getting function call from llm")


" The Pixel Play Credit Card, part of the Born Digital Range of Credit Cards, offers 5% CashBack on choice of any 2 Packs in various categories such as Dining & Entertainment, Travel, Grocery, Electronics, and Fashion. It also offers 3% CashBack on choice of any one e-commerce merchant and 1% Unlimited CashBack on all spends. It can be applied for and controlled through PayZapp. The card also offers customization options for design and billing cycle. The MoneyBack+ Credit Card is another option for everyday spends, offering 5X CashPoints on EMI spends and 1 CashPoint per every Rs.150 on other spends. It also has a festive offer for a lifetime free credit card and vouchers worth up to INR 5,000. The Freedom Credit Card is another option for everyday spends and big purchases, offering 5X CashPoints on EMI spends and 1 CashPoint per every Rs.150 on other spends. It also has a festive offer for a lifetime free credit card and vouchers worth up to INR 5,000. The Pixel Play Credit Card is av

## Function Call Basic using OpenAI

In [14]:
from openai import OpenAI

client = OpenAI()

In [15]:
# step 1 : create function call
function_description=[
    {
        "name": "get_kp_details",
        "description": "Get the details of kp information",
            "parameters": 
            {
                "type": "object",
                "properties": {
                    "parent_name": 
                        {
                            "type": "string",
                            "description": "Name of the parent",
                        
                        },
                },
                "required": ["parent_name"],
            },
    },
    
]


# step 2: make llm call

messages = [
    {"role": "system", "content": "you are helpfull assistent to get information"},
    {"role": "user", "content": "Please provoide parents details"},
    {"role": "assistant", "content": "Hi i can assist you please provide parents details"},
    {"role": "user", "content": "please provoide kp parents details"},
    ]
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    functions=function_description,
    function_call="auto",
    temperature=0.7,
    max_tokens=100   
)

# completion.choices[0].message.function_call.arguments This will give the arguments of function call which we can pass to the function

def get_kp_details(parent_name):
    import json
    parent_info={
        "parent_name":parent_name,
        "Age" : 40,
        "Mother_name":"abc",
        "Father_name":"xyz",
        "Address":"xyz street"
    }
    return json.dumps(parent_info)

# this

import json
kpparent_info=json.loads(completion.choices[0].message.function_call.arguments)
#kpparent_info

parent_function=eval(completion.choices[0].message.function_call.name)
parent_info = parent_function(**kpparent_info)

parent_info #we will get parent details with helo of llm



'{"parent_name": "kp", "Age": 40, "Mother_name": "abc", "Father_name": "xyz", "Address": "xyz street"}'

## Tool Call Basic OpenAI

In [16]:
# Step : 1 set the tools configuration
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_kp_details",
            "description": "This function will get the details of kp information",
            "parameters": {
                "type": "object",
                "properties": {
                    "parent_name": {
                        "type": "string",
                        "description": "The kp parent name"
                    }
                },
                "required": ["parent_name"],
                "additionalProperties": False
            }
        }
    }
]

# step 2: make llm call to get parameters

messages = [
    {"role": "system", "content": "you are helpfull assistent to get information"},
    {"role": "user", "content": "Please provoide parents details"},
    {"role": "assistant", "content": "Hi i can assist you please provide parents details"},
    {"role": "user", "content": "please provoide kp parents details"},
    ]
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    temperature=0.7,
    max_tokens=100   
)

# completion.choices[0].message.function_call.arguments This will give the arguments of function call which we can pass to the function

def get_kp_details(parent_name):
    import json
    parent_info={
        "parent_name":parent_name,
        "Age" : 40,
        "Mother_name":"abc",
        "Father_name":"xyz",
        "Address":"xyz street"
    }
    return json.dumps(parent_info)

#completion.choices[0].message.tool_calls[0].function.arguments 

import json
kpparent_info=json.loads(completion.choices[0].message.tool_calls[0].function.arguments)
#kpparent_info

parent_function=eval(completion.choices[0].message.tool_calls[0].function.name)
parent_info = parent_function(**kpparent_info)

parent_info #we will get parent details with helo of llm

'{"parent_name": "kp", "Age": 40, "Mother_name": "abc", "Father_name": "xyz", "Address": "xyz street"}'

In [22]:
completion.choices[0].message.tool_calls[0].function.name

'get_kp_details'

# Done